In [11]:
import tensorflow as tf
import numpy as np

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('/tmp/data', one_hot = True)

Extracting /tmp/data\train-images-idx3-ubyte.gz
Extracting /tmp/data\train-labels-idx1-ubyte.gz
Extracting /tmp/data\t10k-images-idx3-ubyte.gz
Extracting /tmp/data\t10k-labels-idx1-ubyte.gz


In [23]:
tf.reset_default_graph()

learning_rate = 0.01
dropout_input = 0.8
dropout_hidden = 0.8
batch_size = 256
num_steps = 7000
display_steps = 200
beta = 0

num_input = 28*28
num_hidden1 = 500
num_hidden2 = 500
num_hidden3 = 500
num_layers = 3
num_classes = 10

X = tf.placeholder(tf.float32, [None, num_input])
Y = tf.placeholder(tf.float32, [None, num_classes])

In [24]:
def neural_net(x, weights, biases, dropout_input, dropout_hidden):
    #reshape x back to be an image
    #x = tf.reshape(x, [-1, 28, 28 ,1])
    
    layer1 = tf.matmul(x, weights['w1']) + biases['b1']
    layer1 = tf.contrib.nn.alpha_dropout(tf.nn.selu(layer1), dropout_input)
    
    layer2 = tf.matmul(layer1, weights['w2']) + biases['b2']
    layer2 = tf.contrib.nn.alpha_dropout(tf.nn.selu(layer2), dropout_hidden)
    
    layer3 = tf.matmul(layer2, weights['w3']) + biases['b3']
    layer3 = tf.contrib.nn.alpha_dropout(tf.nn.selu(layer3), dropout_hidden)
    
    output = tf.matmul(layer3, weights['w_out']) + biases['b_out']
    return output

In [25]:
weights = {
    'w1': tf.get_variable('w1', shape=[num_input, num_hidden1], initializer=tf.contrib.layers.variance_scaling_initializer(factor=1.0, mode='FAN_IN')),
    'w2': tf.get_variable('w2', shape=[num_hidden1, num_hidden2], initializer=tf.contrib.layers.variance_scaling_initializer(factor=1.0, mode='FAN_IN')),
    'w3': tf.get_variable('w3', shape=[num_hidden2, num_hidden3], initializer=tf.contrib.layers.variance_scaling_initializer(factor=1.0, mode='FAN_IN')),
    'w_out': tf.get_variable('w_out', shape=[num_hidden3, num_classes], initializer=tf.contrib.layers.variance_scaling_initializer(factor=1.0, mode='FAN_IN'))
}

biases = {
    'b1': tf.get_variable('b1', shape=[num_hidden1], initializer=tf.contrib.layers.variance_scaling_initializer(factor=1.0, mode='FAN_IN')),
    'b2': tf.get_variable('b2', shape=[num_hidden2], initializer=tf.contrib.layers.variance_scaling_initializer(factor=1.0, mode='FAN_IN')),
    'b3': tf.get_variable('b3', shape=[num_hidden3], initializer=tf.contrib.layers.variance_scaling_initializer(factor=1.0, mode='FAN_IN')),
    'b_out': tf.get_variable('b_out', shape=[num_classes], initializer=tf.contrib.layers.variance_scaling_initializer(factor=1.0, mode='FAN_IN'))
}

logits = neural_net(X, weights, biases, dropout_input, dropout_hidden)

regularizer = tf.nn.l2_loss(weights['w1']) + tf.nn.l2_loss(weights['w2']) + tf.nn.l2_loss(weights['w_out'])
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = logits, labels = Y) + beta * regularizer)
optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate)
train_op = optimizer.minimize(loss_op)

correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

init = tf.global_variables_initializer()

In [26]:
with tf.Session() as sess:
    sess.run(init)
    
    for step in range(1, num_steps+1):
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        
        sess.run(train_op, feed_dict = {X: batch_x, Y:batch_y})
        
        if step % display_steps == 0 or step == 1:
            loss, acc = sess.run([loss_op, accuracy], feed_dict = {X: batch_x, Y:batch_y})
            print("Step" + str(step) + ", Minibatch loss" +"{:.4f}".format(loss) + ", Trainning accuracy" + "{:.3f}".format(acc))
    
    print("Optimization done!")
    
    test_accu = sess.run(accuracy, feed_dict = {X: mnist.test.images, Y:mnist.test.labels})
    
    print("Test accuracy:", test_accu)

Step1, Minibatch loss2.0238, Trainning accuracy0.367
Step200, Minibatch loss0.3128, Trainning accuracy0.891
Step400, Minibatch loss0.1298, Trainning accuracy0.965
Step600, Minibatch loss0.1923, Trainning accuracy0.934
Step800, Minibatch loss0.1357, Trainning accuracy0.957
Step1000, Minibatch loss0.2793, Trainning accuracy0.914
Step1200, Minibatch loss0.2754, Trainning accuracy0.949
Step1400, Minibatch loss0.2165, Trainning accuracy0.938
Step1600, Minibatch loss0.0702, Trainning accuracy0.977
Step1800, Minibatch loss0.2064, Trainning accuracy0.961
Step2000, Minibatch loss0.2929, Trainning accuracy0.938
Step2200, Minibatch loss0.2101, Trainning accuracy0.949
Step2400, Minibatch loss0.1467, Trainning accuracy0.953
Step2600, Minibatch loss0.1782, Trainning accuracy0.941
Step2800, Minibatch loss0.0435, Trainning accuracy0.984
Step3000, Minibatch loss0.2079, Trainning accuracy0.953
Step3200, Minibatch loss0.1668, Trainning accuracy0.973
Step3400, Minibatch loss0.1479, Trainning accuracy0.961